# Consumption Optimization

The goal of this notebook is to propose a simple optimization formulation for utility maximization of multiperiod consumption given fixed returns and known income evolution.
When dealing with the asset management problem, it is important not to lose focus on what the ultimate goal is. Very often it is simplified as *maximizing returns and minimizing risk*, but the true purpose goes deeper. That goal is to maximize your wellbeing/happiness. That is very abstract, which is why often it is simplified to returns and risk. However, in this exercise a very simple model is proposed in which we try to maximize the utility of our consumption over our lifetime.
The chosen utility function is the power utility function, and future utility is discounted geometrically to the present. The utility of the consumption over our lifetime thus is

$$ \sum_{i}\beta^{i-1}\mathcal{U}(C) = \sum_{i}\beta^{i-1}\frac{C_{i}^{1-\gamma}}{1-\gamma} $$
Where $ \beta \in (0,1] $ is the temporal discount factor and $ \gamma \in [0,1] $ is the power factor which characterizes the utility function.
Note that $$ \lim_{\gamma \to 1} \mathcal{U}(C) = ln(C) $$

For visualization purposes, here is the value of $ \mathcal{U}(C) $ for different values of $ C$

In [177]:
import plotly.graph_objects as go
import numpy as np

In [178]:
c = np.linspace(1,100000,200)
gammas = np.linspace(.5,.9,5)
traces = []
for gamma in gammas:
    u = c**(1-gamma)/(1-gamma)
    t = go.Scatter(x=c,y=u,name=str(gamma),mode="lines")
    traces.append(t)
fig = go.Figure(traces)
fig.show()

And here we can visualize the discounting of some utility value over time for different $ \beta $ values

In [179]:
u = 10
traces = []
betas = np.linspace(.7,1,10)

for beta in betas:
    u_now = u*beta**np.arange(0,T_WORKING+T_RETIRED)
    t = go.Scatter(x=np.arange(0,T_WORKING+T_RETIRED),y=u_now,name=str(beta),mode="lines")
    traces.append(t)
fig = go.Figure(traces)
fig.show()

The wealth equation will be a very simplified model in which period returns are known and fixed.
$$ W_{t+1} = W_t(1+R) - C_{t+1} + I_{t+1} $$
Where $ I_t $ is income at time t and $$ W_t \geq 0 \quad \forall{t} $$

The income model will also be a very simplified model where it grows linearly until it reaches certain maximum level.
$$ I_{t+1} = \min{(I_t + \Delta I,\bar{I})} $$

The question at hand is thus, what consumption maximizes utility over our lifetime? Is it wiser to spend now or save, invest the money and consume later?
For simplification purposes we will assume we are at the beginning of our career.

In [180]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

In [181]:
BETA = 1
GAMMA = .7

W_0 = 10000
I_0 = 25000
DELTA_I = 2000
I_MAX = 50000

T_WORKING = 35
T_RETIRED = 25

R = 0.07

In [182]:
income = np.zeros(T_WORKING+T_RETIRED)
for i in range(T_WORKING+T_RETIRED):
    if i == 0:
        income[i] = I_0
    elif i < T_WORKING:
        income[i] = min(income[i-1] + DELTA_I,I_MAX)
    else:
        income[i] = 0

wealth = np.zeros(T_WORKING+T_RETIRED)
consumption = np.ones(T_WORKING+T_RETIRED)*10000
x = np.concatenate((wealth.T,consumption.T))
A = np.zeros([T_WORKING+T_RETIRED,2*(T_WORKING+T_RETIRED)])
A[0,0] = -1
A[0,T_WORKING+T_RETIRED] = -1
for i in range(1,T_WORKING+T_RETIRED):
    A[i,i] = -1
    A[i,i-1] = 1+R
    A[i,i+T_WORKING+T_RETIRED] = -1
lb = np.zeros(T_WORKING+T_RETIRED)
lb[0] = -W_0*(1+R)
lb -= income
const_wequation = LinearConstraint(A,lb,lb)
const_pos = LinearConstraint(np.eye(2*(T_WORKING+T_RETIRED)),np.zeros(2*(T_WORKING+T_RETIRED)))
def obj_function(x):
    consumption = x[T_WORKING+T_RETIRED:]
    utility = consumption**(1-GAMMA)/(1-GAMMA)
    discount = BETA**np.arange(0,T_WORKING+T_RETIRED)
    return -discount.T@utility

In [183]:
options = {
    "maxiter":5000
}
sol = minimize(obj_function,x,constraints=[const_wequation,const_pos],options=options)
opt_consumption = sol.x[T_WORKING+T_RETIRED:]
opt_wealth = sol.x[:T_WORKING+T_RETIRED]
print(sol)

C:\Users\Fernando_Celaya\AppData\Local\Temp\ipykernel_16240\3966111218.py:27: RuntimeWarning:

invalid value encountered in power



 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -3927.814240978043
       x: [ 3.824e+03  1.383e+03 ...  2.713e-01  9.990e+03]
     nit: 184
     jac: [ 0.000e+00  0.000e+00 ... -2.492e+00 -1.587e-03]
    nfev: 22342
    njev: 184


In [184]:
t1 = go.Scatter(y=opt_wealth,name="wealth",mode="lines",yaxis='y1')
t2 = go.Bar(y=opt_consumption,name="consumption",yaxis='y2')
t3 = go.Bar(y=income,name="income",yaxis='y2')
data = [t1,t2,t3]
layout = go.Layout(title='Optimal consumption evolution',
                   yaxis=dict(title='Wealth'),
                   yaxis2=dict(title='Consumption and Income',
                               overlaying='y',
                               side='right'))
# fig.add_trace(go.Scatter(y=opt_wealth,name="wealth",mode="lines",yaxis='y1'))
# fig.add_trace(go.Bar(y=opt_consumption,name="consumption",yaxis='y2'))
# fig.add_trace(go.Bar(y=income,name="income",yaxis='y2'))
fig = go.Figure(data=data, layout=layout)
fig.show()

Let's add a minimum consumption for all periods, as it doesn't seem like retirement is going to be much fun like this.

In [185]:
MIN_C = 10000
A = np.zeros([T_WORKING+T_RETIRED,2*(T_WORKING+T_RETIRED)])
for i in range(T_WORKING+T_RETIRED):
    A[i,i+T_WORKING+T_RETIRED] = 1
lb = np.ones(T_WORKING+T_RETIRED)*MIN_C
const_minc = LinearConstraint(A,lb=lb)

In [186]:
options = {
    "maxiter":5000
}
sol = minimize(obj_function,x,constraints=[const_wequation,const_pos,const_minc],options=options)
opt_consumption = sol.x[T_WORKING+T_RETIRED:]
opt_wealth = sol.x[:T_WORKING+T_RETIRED]
print(sol)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -4284.226981474045
       x: [ 8.928e+03  6.329e+03 ...  1.753e+04  2.169e+04]
     nit: 80
     jac: [ 0.000e+00  0.000e+00 ... -1.038e-03 -9.155e-04]
    nfev: 9692
    njev: 80


In [187]:
t1 = go.Scatter(y=opt_wealth,name="wealth",mode="lines",yaxis='y1')
t2 = go.Bar(y=opt_consumption,name="consumption",yaxis='y2')
t3 = go.Bar(y=income,name="income",yaxis='y2')
data = [t1,t2,t3]
layout = go.Layout(title='Optimal consumption evolution',
                   yaxis=dict(title='Wealth'),
                   yaxis2=dict(title='Consumption and Income',
                               overlaying='y',
                               side='right'))
# fig.add_trace(go.Scatter(y=opt_wealth,name="wealth",mode="lines",yaxis='y1'))
# fig.add_trace(go.Bar(y=opt_consumption,name="consumption",yaxis='y2'))
# fig.add_trace(go.Bar(y=income,name="income",yaxis='y2'))
fig = go.Figure(data=data, layout=layout)
fig.show()

As we can see, the shape of the wealth distribution does not change much. However, to fulfill the minimum consumption constraint we should start saving a little bit before retirement.

Any of these solutions are however quite counterintuitive. Spending everything in your working years and then surviving your retirment on the minimum doesn't sound optimal at all. In reality, in retirement we have more free time so we probably can consume more, as we age we can do less activities so consumption presumably has less utility, we probably don't want too spend to many years living on the minimum and would like to splurge every now and then even if that means having spent a little bit less before.
This shows how solutions can only be as good as the problems we formulate, and modeling human desires and behaviours in a mathematical problem isn't as easy as it sounds.